In [71]:
import urllib.request, urllib.error, urllib.parse
import csv
from urllib.request import build_opener, HTTPCookieProcessor, Request

#PUT JOURNAL URL HERE
url = 'https://journals.aom.org/toc/amr/45/3'

opener = build_opener(HTTPCookieProcessor())
request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
response = opener.open(request, timeout=30)
allContent = str(response.read())

In [72]:
def toSheetHeader(name):
    # field names  
    fields = ['Title', 'Abstract','Journal']  
    
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'w') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the fields  
        csvwriter.writerow(fields)  

In [73]:
def toSheet(rows,name):
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'a') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the data rows  
        csvwriter.writerows(rows) 

In [74]:
allarticles = []

#GETTING JOURNAL TITLE
journal_index = allContent.find('<title')
journal_index = allContent.find('>',journal_index) + 1
journal = ""
while(allContent[journal_index]!="|"):
    journal = journal + allContent[journal_index]
    journal_index = journal_index+1

toSheetHeader(journal)
                    
#FINDING THE LINK TO THE ISSUE, allContent is all of the code for the original URL                    
issue_index = 0
while(allContent.find('class="issue-info"',issue_index,len(allContent))!=-1 and 
      allContent.find('VOL. 5',allContent.find('class="issue-info"',issue_index,len(allContent)),len(allContent))!=-1):
      
    issue_index = allContent.find('class="issue-info"',issue_index,len(allContent))
      
    issue_index = issue_index-30
      
    issue_link_index = allContent.find("href=",issue_index,len(allContent))
    issue_link = ""
    i = issue_link_index+6  #change this number based on how long until the link from issue_link_index
                                  
    while(allContent[i]!='"'):
        issue_link = issue_link+allContent[i]
        i = i+1
    
    #getting code for ISSUE LINK here
    opener = build_opener(HTTPCookieProcessor())
    request = Request("https://journals.aom.org"+issue_link, headers={'User-Agent': 'Mozilla/5.0'})
    response = opener.open(request, timeout=30)
    webContent = str(response.read())

    
    #finding article title and the article link
    article_index = 0   
    while(webContent.find('class="issue-item__title"',article_index,len(webContent))!=-1):
        currarticle = []
        article_index = webContent.find('class="issue-item__title"',article_index,len(webContent))
    
      
        #link_index = webContent.find("href=",article_index,len(webContent))

        #link = ""
        #i = link_index+6
        
        #while(webContent[i]!='"'):
            #link = link+webContent[i]
            #i = i+1
            
        #i = i + 2
        #webContent.find("",i,len(webContent))

        title = ""
      
        article_index = webContent.find('title="', article_index, len(webContent))
        i = article_index + 7
        while(webContent[i]!='"'):
            title = title+webContent[i]
            i = i+1

        title = title.replace("\\n"," ")
        currarticle.append(title)
        #print("\033[1m",title,"\033[0;0m")

        
        execute = True
        
        abstract_index = webContent.find('accordion__content toc-item__abstract',article_index,len(webContent))
        if (abstract_index == -1):
            abstract_index = webContent.find('<div class="abstract">')
        if (abstract_index == -1):
            abstract_index = webContent.find('<div class="section1')
        if( abstract_index == -1):
            execute = False
        #if (abstract_index == -1):
            #abstract_index = articleContent.find('YOU CHANGE THIS')
            
        if(execute):    
            #abstract_index = articleContent.find("<p",abstract_index,len(articleContent))
            abstract_index = webContent.find(">",abstract_index,len(webContent))
            i = abstract_index+1
      
            end_abstract_index = webContent.find("</div>",abstract_index,len(webContent))

            abstract = ""

            while(i!=end_abstract_index):
                abstract = abstract+webContent[i]
                i = i + 1
    
            abstract = abstract.replace("\\n"," ")
            currarticle.append(abstract)

        currarticle.append(journal)
        allarticles.append(currarticle)
        toSheet(allarticles,journal)
        allarticles=[]
        article_index = article_index+1
        issue_index = issue_index+50


KeyboardInterrupt: 